* Extract features using pre-trained Inception-V3 model

In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys, os, time, gc
import requests, shutil
from skimage import io
import keras
from keras.applications import InceptionV3

%matplotlib inline

Using TensorFlow backend.


# Resized Images Info

In [3]:
# Read train and test dataframes
df_train = pd.read_csv('./data/resized/train_resized.csv')
df_test = pd.read_csv('./data/resized/test_resized.csv')

print('Train:\t\t', df_train.shape)
print('Test:\t\t', df_test.shape)

print('Landmarks:\t', len(df_train['landmark_id'].unique()))

Train:		 (1223295, 3)
Test:		 (117224, 2)
Landmarks:	 14950


# Feature Extraction using Pretrained CNN Model

In [5]:
# Use pre-trained Inception V3 model from Keras
inception = InceptionV3(include_top=False, weights='imagenet', input_shape=(256, 256, 3), pooling='avg')

inception.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 127, 127, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 127, 127, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [6]:
# Define train_imgs and test_imgs
train_imgs = np.zeros(shape=(len(df_train), 2048), dtype=np.float32)
test_imgs = np.zeros(shape=(len(df_test), 2048), dtype=np.float32)

In [7]:
# Process training images
steps = 50000
for i in range(0, len(df_train), steps):
    tmp_imgs = []
    print('\nProcess: {:10d}'.format(i))
    
    start = i
    end = min(len(df_train), i + steps)
    for idx in range(start, end):
        if idx % 1000 == 0:
            print('=', end='')
        img = io.imread('./data/resized/train_resized/' + str(idx) + '.jpg')
        tmp_imgs.append(img)
        
    tmp_imgs = np.array(tmp_imgs, dtype=np.float32) / 255.0
    tmp_prediction = inception.predict(tmp_imgs)
    train_imgs[start: end, ] = tmp_prediction
    _ = gc.collect()


Process:          0
Process:      50000
Process:     100000
Process:     150000
Process:     200000
Process:     250000
Process:     300000
Process:     350000
Process:     400000
Process:     450000
Process:     500000
Process:     550000
Process:     600000
Process:     650000
Process:     700000
Process:     750000
Process:     800000
Process:     850000
Process:     900000
Process:     950000
Process:    1000000
Process:    1050000
Process:    1100000
Process:    1150000
Process:    1200000

In [36]:
# Process test images
steps = 20000
for i in range(0, len(df_test), steps):
    tmp_imgs = []
    print('\nProcess: {:10d}'.format(i))
    
    start = i
    end = min(len(df_test), i + steps)
    for idx in range(start, end):
        if idx % 1000 == 0:
            print('=', end='')
        img = io.imread('./data/resized/test_resized/' + str(idx) + '.jpg')
        tmp_imgs.append(img)
        
    tmp_imgs = np.array(tmp_imgs, dtype=np.float32) / 255.0
    tmp_prediction = inception.predict(tmp_imgs)
    test_imgs[start: end, ] = tmp_prediction
    _ = gc.collect()


Process:          0
Process:      20000
Process:      40000
Process:      60000
Process:      80000
Process:     100000

In [43]:
# Save to disk
np.save('./data/resized/train_features.npy', train_imgs)
np.save('./data/resized/test_features.npy', test_imgs)